In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, LSTM, GRU
from sklearn.utils import resample
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load the dataset
df = pd.read_csv('combodataset.csv', header = None)

df.columns = ['age', 'sex', 'cp', 'trestbps', 'chol',
              'fbs', 'restecg', 'thalach', 'exang', 
              'oldpeak', 'slope', 'target']

df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,target
0,40,1,2,140,289,0,0,172,0,0.0,1,0
1,49,0,3,160,180,0,0,156,0,1.0,2,1
2,37,1,2,130,283,0,1,98,0,0.0,1,0
3,48,0,4,138,214,0,0,108,1,1.5,2,1
4,54,1,3,150,195,0,0,122,0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1185,45,1,1,110,264,0,0,132,0,1.2,2,1
1186,68,1,4,144,193,1,0,141,0,3.4,2,1
1187,57,1,4,130,131,0,0,115,1,1.2,2,1
1188,57,0,2,130,236,0,2,174,0,0.0,2,1


In [3]:
# Separate features and target variable
X = df.drop('target', axis=1)
y = df['target']


In [4]:

# 1. Load and preprocess the data
# Assuming you have your data in the X and y variables

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [5]:
input_shape = (X_train.shape[1], 1)
input_shape_cnn = (X_train.shape[1], 1)

In [6]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [7]:
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(filters=11, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [10]:
# Train the individual models
cnn_model = build_cnn_model(input_shape)
cnn_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
12/12 [==============================] - 2s 38ms/step - loss: 0.5979 - accuracy: 0.6975 - val_loss: 0.5846 - val_accuracy: 0.7765
Epoch 2/100
12/12 [==============================] - 0s 14ms/step - loss: 0.4955 - accuracy: 0.7613 - val_loss: 0.5174 - val_accuracy: 0.8034
Epoch 3/100
12/12 [==============================] - 0s 14ms/step - loss: 0.4849 - accuracy: 0.7866 - val_loss: 0.5049 - val_accuracy: 0.7966
Epoch 4/100
12/12 [==============================] - 0s 14ms/step - loss: 0.4605 - accuracy: 0.7966 - val_loss: 0.5283 - val_accuracy: 0.7244
Epoch 5/100
12/12 [==============================] - 0s 14ms/step - loss: 0.4462 - accuracy: 0.8067 - val_loss: 0.5032 - val_accuracy: 0.7933
Epoch 6/100
12/12 [==============================] - 0s 14ms/step - loss: 0.4368 - accuracy: 0.8252 - val_loss: 0.4971 - val_accuracy: 0.8000
Epoch 7/100
12/12 [==============================] - 0s 13ms/step - loss: 0.4120 - accuracy: 0.8134 - val_loss: 0.4530 - val_accuracy: 0.8252
Epoch 

In [11]:
# 8. Evaluate the model
test_loss, test_accuracy = cnn_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


19/19 [==============================] - 0s 3ms/step - loss: 0.3623 - accuracy: 0.8555
Test Loss: 0.3623
Test Accuracy: 0.8555


In [12]:
# Make predictions
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


19/19 [==============================] - 0s 3ms/step
Confusion Matrix:
[[231  44]
 [ 42 278]]

Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.84      0.84       275
           1       0.86      0.87      0.87       320

    accuracy                           0.86       595
   macro avg       0.85      0.85      0.85       595
weighted avg       0.86      0.86      0.86       595



In [13]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [14]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(11, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [15]:
lstm_model = build_lstm_model(input_shape)
lstm_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
              callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
12/12 [==============================] - 13s 262ms/step - loss: 0.6375 - accuracy: 0.6756 - val_loss: 0.6611 - val_accuracy: 0.7076
Epoch 2/100
12/12 [==============================] - 1s 43ms/step - loss: 0.5915 - accuracy: 0.6941 - val_loss: 0.6612 - val_accuracy: 0.6555
Epoch 3/100
12/12 [==============================] - 1s 44ms/step - loss: 0.5835 - accuracy: 0.6992 - val_loss: 0.6665 - val_accuracy: 0.6420
Epoch 4/100
12/12 [==============================] - 1s 44ms/step - loss: 0.5739 - accuracy: 0.7042 - val_loss: 0.6702 - val_accuracy: 0.5378
Epoch 5/100
12/12 [==============================] - 1s 44ms/step - loss: 0.5679 - accuracy: 0.7176 - val_loss: 0.6655 - val_accuracy: 0.6336
Epoch 6/100
12/12 [==============================] - 1s 44ms/step - loss: 0.5609 - accuracy: 0.7210 - val_loss: 0.6675 - val_accuracy: 0.5395
Epoch 7/100
12/12 [==============================] - 1s 44ms/step - loss: 0.5615 - accuracy: 0.7261 - val_loss: 0.6702 - val_accuracy: 0.6252
Epoc

12/12 [==============================] - 0s 42ms/step - loss: 0.1722 - accuracy: 0.9261 - val_loss: 0.4041 - val_accuracy: 0.8521
Epoch 59/100
12/12 [==============================] - 0s 42ms/step - loss: 0.2065 - accuracy: 0.9160 - val_loss: 0.4302 - val_accuracy: 0.8454
Epoch 60/100
12/12 [==============================] - 1s 48ms/step - loss: 0.2128 - accuracy: 0.9210 - val_loss: 0.4436 - val_accuracy: 0.8353


In [16]:
# 8. Evaluate the model
test_loss, test_accuracy = lstm_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


19/19 [==============================] - 0s 10ms/step - loss: 0.3805 - accuracy: 0.8420
Test Loss: 0.3805
Test Accuracy: 0.8420


In [17]:
# Make predictions
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


19/19 [==============================] - 3s 10ms/step
Confusion Matrix:
[[233  42]
 [ 52 268]]

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.85      0.83       275
           1       0.86      0.84      0.85       320

    accuracy                           0.84       595
   macro avg       0.84      0.84      0.84       595
weighted avg       0.84      0.84      0.84       595



In [18]:
# 4. Hyperparameter tuning
learning_rate = 0.001
batch_size = 50
epochs = 100



# 5. Learning rate scheduling
lr_scheduler = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate, decay_steps=10000, decay_rate=0.9
)
optimizer = keras.optimizers.Adam(learning_rate=lr_scheduler)



# 6. Regularization techniques
callback_early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)
callback_checkpoint = keras.callbacks.ModelCheckpoint(
    'model_checkpoint.h5', save_best_only=True
)



In [19]:
def build_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(11, return_sequences=True, input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(32, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(64, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128, return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(GRU(128))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Output layer
    
    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [20]:
gru_model = build_gru_model(input_shape)
gru_model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test),
             callbacks=[callback_early_stopping, callback_checkpoint])


Epoch 1/100
12/12 [==============================] - 19s 348ms/step - loss: 0.6520 - accuracy: 0.6471 - val_loss: 0.6953 - val_accuracy: 0.4622
Epoch 2/100
12/12 [==============================] - 1s 82ms/step - loss: 0.5879 - accuracy: 0.7210 - val_loss: 0.7069 - val_accuracy: 0.4622
Epoch 3/100
12/12 [==============================] - 1s 83ms/step - loss: 0.5197 - accuracy: 0.7580 - val_loss: 0.7186 - val_accuracy: 0.4622
Epoch 4/100
12/12 [==============================] - 1s 82ms/step - loss: 0.4882 - accuracy: 0.7647 - val_loss: 0.7302 - val_accuracy: 0.4622
Epoch 5/100
12/12 [==============================] - 1s 82ms/step - loss: 0.4818 - accuracy: 0.7832 - val_loss: 0.7402 - val_accuracy: 0.4622
Epoch 6/100
12/12 [==============================] - 1s 82ms/step - loss: 0.4617 - accuracy: 0.7815 - val_loss: 0.7526 - val_accuracy: 0.4622
Epoch 7/100
12/12 [==============================] - 1s 82ms/step - loss: 0.4663 - accuracy: 0.7866 - val_loss: 0.7767 - val_accuracy: 0.4622
Epoc

In [21]:
# 8. Evaluate the model
test_loss, test_accuracy = gru_model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')


19/19 [==============================] - 0s 16ms/step - loss: 0.6953 - accuracy: 0.4622
Test Loss: 0.6953
Test Accuracy: 0.4622


In [24]:
# Make predictions
y_pred = gru_model.predict(X_test)
y_pred_classes = np.round(y_pred)


# Calculate confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report


conf_matrix = confusion_matrix(y_test, y_pred_classes)
class_report = classification_report(y_test, y_pred_classes)

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)


19/19 [==============================] - 0s 15ms/step
Confusion Matrix:
[[275   0]
 [320   0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.46      1.00      0.63       275
           1       0.00      0.00      0.00       320

    accuracy                           0.46       595
   macro avg       0.23      0.50      0.32       595
weighted avg       0.21      0.46      0.29       595



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
# Make predictions using individual models
cnn_predictions = (cnn_model.predict(X_test) > 0.5).astype("int32")
lstm_predictions = (lstm_model.predict(X_test) > 0.5).astype("int32")
gru_predictions = (gru_model.predict(X_test) > 0.5).astype("int32")



19/19 [==============================] - 0s 16ms/step


In [30]:
# Combine predictions using a majority voting ensemble
ensemble_predictions = np.mean([cnn_predictions, lstm_predictions, gru_predictions], axis=0) >= 0.5
ensemble_predictions = ensemble_predictions.astype("int32")

In [31]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8420


In [32]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, ensemble_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, ensemble_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")


Confusion Matrix:
[[250  25]
 [ 69 251]]
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.91      0.84       275
           1       0.91      0.78      0.84       320

    accuracy                           0.84       595
   macro avg       0.85      0.85      0.84       595
weighted avg       0.85      0.84      0.84       595

Ensemble Accuracy: 0.8420


In [33]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.utils import resample
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import cross_val_score

In [34]:
# List of base models

lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(X_train, y_train)

svm_l_model = SVC(kernel="linear")
svm_l_model.fit(X_train, y_train)

svm_p_model = SVC(kernel="poly")
svm_p_model.fit(X_train, y_train)

svm_r_model = SVC(kernel="rbf")
svm_r_model.fit(X_train, y_train)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)


knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

ab_model = AdaBoostClassifier(n_estimators=100, random_state=42)
ab_model.fit(X_train, y_train)

xgb_model = XGBClassifier(n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)

gpc_model = GaussianProcessClassifier(random_state=42)
gpc_model.fit(X_train, y_train)

GaussianProcessClassifier(random_state=42)

In [35]:
lr_predictions = lr_model.predict(X_test)
svm_l_predictions = svm_l_model.predict(X_test)
svm_p_predictions = svm_p_model.predict(X_test)
svm_r_predictions = svm_r_model.predict(X_test)
rf_predictions = rf_model.predict(X_test)
knn_predictions = knn_model.predict(X_test)
gb_predictions = gb_model.predict(X_test)
ab_predictions = ab_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
gpc_predictions = gpc_model.predict(X_test)


In [36]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, lr_predictions, 
                                        svm_l_predictions, svm_p_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        ab_predictions, xgb_predictions, gpc_predictions))


In [37]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

In [38]:
# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.8958


In [39]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Confusion Matrix:
[[242  33]
 [ 29 291]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.88      0.89       275
           1       0.90      0.91      0.90       320

    accuracy                           0.90       595
   macro avg       0.90      0.89      0.90       595
weighted avg       0.90      0.90      0.90       595

Ensemble Accuracy: 0.8958


In [40]:
# Combine predictions using ensemble (Voting Classifier)
ensemble_predictions = np.column_stack((cnn_predictions, lstm_predictions, gru_predictions, svm_r_predictions, 
                                        rf_predictions, knn_predictions, gb_predictions, 
                                        xgb_predictions, gpc_predictions))


In [41]:
# Perform majority voting to get the final prediction
final_predictions = np.mean(ensemble_predictions, axis=1) >= 0.5
final_predictions = final_predictions.astype("int32")

# Calculate accuracy of the ensemble
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, final_predictions)

print("Confusion Matrix:")
print(conf_matrix)

# Print classification report
class_report = classification_report(y_test, final_predictions)
print("Classification Report:")
print(class_report)

# Evaluate ensemble accuracy
ensemble_accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Accuracy: {ensemble_accuracy:.4f}")

Ensemble Accuracy: 0.9008
Confusion Matrix:
[[245  30]
 [ 29 291]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89       275
           1       0.91      0.91      0.91       320

    accuracy                           0.90       595
   macro avg       0.90      0.90      0.90       595
weighted avg       0.90      0.90      0.90       595

Ensemble Accuracy: 0.9008
